In [1]:
#NOTE: use paimg9 env
import sys
import os
import numpy as np
import openslide
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg')
import pandas as pd
import warnings
import torch
import torch.nn as nn

from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import Subset, ConcatDataset
import torch.optim as optim
from pathlib import Path

sys.path.insert(0, '../Utils/')
from Utils import create_dir_if_not_exists
# from Utils import generate_deepzoom_tiles, extract_tile_start_end_coords, get_map_startend
# from Utils import get_downsample_factor
# from Utils import minmax_normalize, count_label
# from Utils import log_message, set_seed
# from Eval import compute_performance, plot_LOSS, compute_performance_each_label, get_attention_and_tileinfo
# from train_utils import pull_tiles, get_feature_label_array_dynamic
# from train_utils import ModelReadyData_diffdim, convert_to_dict, prediction
# from Model import Mutation_MIL_MT
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
####################################
######      USERINPUT       ########
####################################
ALL_LABELS = ["AR","MMR (MSH2, MSH6, PMS2, MLH1, MSH3, MLH3, EPCAM)2","PTEN","RB1","TP53","TMB_HIGHorINTERMEDITATE","MSI_POS"]
TUMOR_FRAC_THRES = 0.9 
feature_extraction_method = 'uni1' #retccl, uni1
N_FEATURE = 1024
    
##################
###### DIR  ######
##################
proj_dir = '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/'
folder_name_overlap = "IMSIZE250_OL100"
folder_name_nonoverlap = "IMSIZE250_OL0"
feature_path_opx_train = outdir =  os.path.join(proj_dir + 'intermediate_data/5_model_ready_data', "OPX", folder_name_overlap, 'feature_' + feature_extraction_method, 'TFT' + str(TUMOR_FRAC_THRES))
feature_path_opx_test = outdir =  os.path.join(proj_dir + 'intermediate_data/5_model_ready_data', "OPX", folder_name_nonoverlap, 'feature_' + feature_extraction_method, 'TFT' + str(TUMOR_FRAC_THRES))

In [3]:
################################################
#Create output dir
################################################
folder_name = feature_extraction_method + '/TrainOL100_TestOL0_TFT' + str(TUMOR_FRAC_THRES)  + "/"
outdir =  os.path.join(proj_dir + 'intermediate_data/6_Train_TEST_IDS', 
                       folder_name)
create_dir_if_not_exists(outdir)


##################
#Select GPU
##################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/6_Train_TEST_IDS/uni1/TrainOL100_TestOL0_TFT0.9/' created.
cuda:0


In [4]:
################################################
#    Load OPX 
################################################
opx_data_ol100 = torch.load(feature_path_opx_train + '/OPX_data.pth')
opx_ids_ol100 = torch.load(feature_path_opx_train + '/OPX_ids.pth')
opx_info_ol100  = torch.load(feature_path_opx_train + '/OPX_info.pth')

opx_data_ol0 = torch.load(feature_path_opx_test + '/OPX_data.pth')
opx_ids_ol0 = torch.load(feature_path_opx_test + '/OPX_ids.pth')
opx_info_ol0  = torch.load(feature_path_opx_test + '/OPX_info.pth')

In [5]:
############################################################################################################
#Get Train and test IDs, 80% - 20%
############################################################################################################
opx_ids = opx_ids_ol100
print('total ID:',len(opx_ids))

train_test_valid_df = pd.DataFrame({'SAMPLE_ID': opx_ids})

n_splits = 5 # Number of folds
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42) #Initialize KFold
for SELECTED_FOLD in range(n_splits):
    # Generate the folds
    train_ids_folds = []
    test_ids_folds = []
    for fold, (train_index, test_index) in enumerate(kf.split(opx_ids)):
        train_ids_folds.append([opx_ids[i] for i in train_index])
        test_ids_folds.append([opx_ids[i] for i in test_index])
    
    full_train_ids = train_ids_folds[SELECTED_FOLD]
    test_ids = test_ids_folds[SELECTED_FOLD]
    
    # Randomly select 5% of the train_ids for validation
    train_ids, val_ids = train_test_split(full_train_ids, test_size=0.05, random_state=42)

    #Update dataframe
    cond = train_test_valid_df['SAMPLE_ID'].isin(train_ids)
    train_test_valid_df.loc[cond, 'FOLD' + str(SELECTED_FOLD)] = 'TRAIN'
    cond = train_test_valid_df['SAMPLE_ID'].isin(test_ids)
    train_test_valid_df.loc[cond, 'FOLD' + str(SELECTED_FOLD)] = 'TEST'
    cond = train_test_valid_df['SAMPLE_ID'].isin(val_ids)
    train_test_valid_df.loc[cond, 'FOLD' + str(SELECTED_FOLD)] = 'VALID'

total ID: 203


In [6]:
#This new added OPX must in train or test, because at this point, we want to have more MSI to test
train_must = ['OPX_207','OPX_209','OPX_213','OPX_214','OPX_215']
test_must = ['OPX_208','OPX_210','OPX_211','OPX_212','OPX_216']
train_test_valid_df.loc[train_test_valid_df['SAMPLE_ID'].isin(train_must), ['FOLD0','FOLD1','FOLD2','FOLD3','FOLD4']] = 'TRAIN'
train_test_valid_df.loc[train_test_valid_df['SAMPLE_ID'].isin(test_must), ['FOLD0','FOLD1','FOLD2','FOLD3','FOLD4']] = 'TEST'

In [7]:
train_test_valid_df.to_csv(outdir + 'train_test_split.csv')